</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode
from methods.llm import get_groq_response

# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None, get_response=get_groq_response)
node.get_test_cases(2)
node.evolve("i1", replace=True)

input_dict = {"name": "Dilireba"}
output_dict = node(input_dict) 

Could not load vllm class, check CUDA support and GPU RAM size


Generating test cases: 5case [00:01,  2.60case/s]             


--- Generated 5 test cases
Error in parsing LLM response: 
 JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1) 
Response: 
 To find the birthdate of Leonardo DiCaprio and calculate his age, we'll use a combination of the `webbrowser` and `datetime` libraries in Python to retrieve his birthdate from Wikipedia.

```python
import webbrowser
import datetime

def get_celebrity_birthdate():
    # Search for Leonardo DiCaprio's birthdate on Wikipedia
    query = f"https://en.wikipedia.org/wiki/Leonardo_DiCaprio"
    webbrowser.open(query, new=2)  # Open the link in a new tab/window
    
    # Unfortunately, there's no way to scrape a website directly using a library
    # without getting into hairy web scraping laws. Here's a hypothetical example
    # for demonstration purposes. Please replace this with the actual scraping
    # if needed.
    birthdate_str = "November 11, 1974"
    
    # Process the birthdate string
    birthda

#### Direct Loading Node

In [1]:
from methods.eoh_evolution import EvolNode 

node = EvolNode.load("get_celeb_age")
input_dict = {"name": "Dilireba"}
node(input_dict)

Could not load vllm class, check CUDA support and GPU RAM size


{'age': 31}

#### Alignment-Check for Evaluation

In [2]:
from methods.llm import get_groq_response 

# Evaluation base on alignment check
node.get_response = get_groq_response 
structure_fitness, function_fitness, issue_summary = node._evaluate_fitness(max_tries=1, num_runs=2)

Evaluating fitness: 100%|██████████| 4/4 [00:13<00:00,  3.39s/it]


In [3]:
print(issue_summary)

--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.



In [5]:
# how to put a node into another nodes? 
# Ideally I would then ask for the node (which likely fails on many of the test-cases) to come-up with a code, where it could call on some 'pseudo-node'
# These pseudo-nodes are used, very similar to a pseudo-code 




</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.meta_prompt import MetaPlan, extract_json_from_text
from methods.llm import get_openai_response
from methods.diagram import visualize_plan_dict

# Breaking Down EvolGraph.generate() function components here 

# Step 1: Generate Plan Dict
from methods.llm import get_groq_response 
mp = MetaPlan("Get the age of celebrity.")

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.llm import get_claude_response
from methods.meta_prompt import extract_python_code

prompt = mp._get_prompt_i1_pseudo_code() # Pseudo-Code Prompt (Non-implemented functional)

response = get_claude_response(prompt) # Use Strong LLM to build up pseudo-code

code = extract_python_code(response) # Extract Python Code from response 




In [5]:
# Get Nodes Prompt

graph_prompt = mp._get_prompt_i1_with_pseudo_code(code)

In [7]:
plan_response = get_claude_response(graph_prompt)

In [10]:
# print(graph_prompt) # Input and Output should be specified at least ... currently it's not ...


In [11]:
print(plan_response)

{The intuition is to fetch a celebrity's birth date, get the current date, and calculate the age by comparing these dates, while handling potential errors or missing information.}

```json
{
  "nodes": [
    {
      "task": "Get celebrity name from user input",
      "name": "get_celebrity_name",
      "inputs": [],
      "input_types": [],
      "outputs": ["celebrity_name"],
      "output_types": ["str"],
      "target": "Obtain the name of the celebrity",
      "mode": "CODE"
    },
    {
      "task": "Retrieve celebrity birth date",
      "name": "get_celebrity_birth_date",
      "inputs": ["celebrity_name"],
      "input_types": ["str"],
      "outputs": ["birth_date"],
      "output_types": ["datetime.date"],
      "target": "Fetch the birth date of the celebrity",
      "mode": "CODE"
    },
    {
      "task": "Get current date",
      "name": "get_current_date",
      "inputs": [],
      "input_types": [],
      "outputs": ["current_date"],
      "output_types": ["datetime.da

In [31]:
from methods.llm import get_groq_response

response = get_groq_response(graph_prompt)
print(response)









{To achieve the goal of getting the age of a celebrity, our tactics will involve orchestrating two main steps: retrieving the celebrity's birth date from a database or API and obtaining the current date, then calculating the age by comparing these two dates. We will use a combination of direct programming and user input (via prompts) to enable flexibility in the solution.}

Here's a revised JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal:

```json
{
  "nodes": [
    {
      "task": "Get celebrity's name from user",
      "name": "get_celebrity_info",
      "inputs": ["celebrity_name"],
      "input_types": ["str"],
      "outputs": ["celebrity_name", "birth_date"],
      "output_types": ["str", "datetime.date"],
      "target": "Calculate age of the celebrity",
      "mode": "PROMPT"
    },
    {
      "task": "Get current date",
      "name": "get_current_date",
      "inputs": [],
      "input_types": [],
      "outputs": ["current_date"],
      "out

In [8]:
prompt = mp._get_prompt_i1()

# print(prompt)

from methods.llm import get_groq_response 
response = get_groq_response(prompt)


print(response)

Here's a plan to get Dilireba's age using a Directed Acyclic Graph (DAG):

{ 
  Plan: Identify Dilireba, extract her birthdate from reliable sources, calculate her age.
}

```json
{
  "nodes": [
    {
      "task": "Identify Dilireba",
      "name": "identify_dilireba",
      "inputs": [],
      "input_types": [],
      "outputs": ["Dilireba"],
      "output_types": ["str"],
      "target": "Get Dilireba's age",
      "mode": "PROMPT"
    },
    {
      "task": "Find Dilireba's birthdate",
      "name": "find_birthdate",
      "inputs": ["Dilireba"],
      "input_types": ["str"],
      "outputs": ["birthdate"],
      "output_types": ["date"],
      "target": "Get Dilireba's age",
      "mode": "PROMPT"
    },
    {
      "task": "Calculate age",
      "name": "calculate_age",
      "inputs": ["birthdate"],
      "input_types": ["date"],
      "outputs": ["age"],
      "output_types": ["int"],
      "target": "Get Dilireba's age",
      "mode": "CODE"
    },
    {
      "task": "Format 

Evaluation Node is critical here

In [2]:
# Eval Node 
get_response = get_openai_response 


prompt_content = mp._get_eval_prompt_i1()
response = get_response(prompt_content)
eval_dict = extract_json_from_text(response)


# eval_prompt = MetaPrompt(
#     task=node.get("task"),
#     func_name=node.get("name"),
#     inputs=node.get("input"),
#     outputs=node.get("output"),
#     mode=node.get("mode").lower()
# )
# eval_node = EvolNode(meta_prompt=eval_prompt)

In [3]:
print(prompt_content)

Goal: Evaluating wether the goal Help me date Dilireba, I'm a white colar worker living on salary. has been achieved.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON response describing a task to evaluate whehter the goal is completed.

**Output Format:**

Provide the output in the following JSON structure:
```json
{
    "task": "Evaluate Task",
    "name": "eval_goal",
    "input": "Inputs required for evaluation",
    "output": "Outputs required for evaluation",
    "target": "Purpose of the evaluation",
    "mode: "CODE"
}



In [ ]:
nodes = {}
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(
        task=node.get("task"),
        func_name=node.get("name"),
        inputs=node.get("inputs"),
        outputs=node.get("outputs"),
        input_types=node.get("input_types"),
        output_types=node.get("output_types"),
        mode=node.get("mode").lower()
    )
    nodes[node.get("name")] = EvolNode(meta_prompt=node_prompt)

edges = plan_dict["edges"]

In [4]:
from methods.eoh_evolution import EvolGraph

eg = EvolGraph.read_from_dict(plan_dict)
# eg.evolve()

In [7]:
eg.generate()

TypeError: EvolGraph.generate() missing 1 required positional argument: 'goal'